In [1]:
!pip install langchain chromadb langchain-chroma


In [13]:
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.prompts.chat import HumanMessagePromptTemplate
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_text_splitters import CharacterTextSplitter
from langchain.schema import SystemMessage
from langchain.document_loaders import PyPDFLoader
import os

In [3]:
# Initialize the OpenAI API key and LLM model
OPENAI_API_KEY = ''
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, openai_api_key=OPENAI_API_KEY)

In [4]:
pdf_file_path = r"C:\Users\sarfarazuddin.s\Downloads\The Data Science Handbook.pdf"
if not os.path.exists(pdf_file_path):
    raise FileNotFoundError(f"The file {pdf_file_path} does not exist.")

# Load and split the PDF document into chunks
document_loader = PyPDFLoader(pdf_file_path)
# documents = document_loader.load_and_split()
documents = document_loader.load()

In [5]:
# documents

In [6]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [7]:
# text_chunks

In [14]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [15]:
# Initialize Chroma vector database
persist_directory = r"D:\vectorstore11"  # Replace with your actual persistence directory path
db = Chroma.from_documents(
    # documents=documents,
    documents=text_chunks,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [16]:
# # Set up Chroma retriever
retriever = db.as_retriever()

# Initialize the RetrievalQA chain with your LLM and the Chroma retriever
db_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # We can use 'stuff', 'map_reduce', or 'refine' depending on your needs
    retriever=retriever
)

In [17]:
def retrieve_from_db(query: str) -> str:
    try:
        db_context = db_chain(query)
        db_context = db_context['result'].strip()
        return db_context
    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred while retrieving data from the database."

def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a professional representative of The Data Science Handbook.
        Answer questions related to data science topics, methodologies, and insights from the book. Provide concise and accurate information based on the content of the book.
        If question is asked which is not related to Data Science dont give answer and reply that you are not allowed to give the answers which are not related
        to Data Science.
        """

    
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
        SystemMessage(content=system_message),
        human_qry_template.format(human_input=query, db_context=db_context)
    ]
    try:
        # Get the response from the LLM using the invoke method
        response = llm.invoke(messages).content
        return response
    except Exception as e: 
        print(f"Error: {e}")
        return "An error occurred while generating the response."

# Example of usage
query_input = input("Enter your query: ")

response_data = generate(query_input)
print(f"Response: {response_data}")

Enter your query:  What is Data Science


Response: Data Science is a multidisciplinary field that merges creativity and analytical rigor to extract actionable intelligence from both structured and unstructured data. It involves advanced statistical methods, machine learning, and data analysis to uncover insights that inform decision-making. The field balances the creative aspect of identifying patterns and generating hypotheses with a solid theoretical foundation and rigorous problem-solving skills. Data Scientists aim to efficiently extract actionable intelligence from data, connecting disparate sources and applying innovative problem-solving techniques. This dynamic field drives informed decision-making across various industries.


In [18]:
# Example of usage
query_input = input("Enter your query: ")

response_data = generate(query_input)
print(f"Response: {response_data}")

Enter your query:  What is Physics


Response: I'm not allowed to give answers which are not related to Data Science.
